**Author:** Kimberly Alexander  
**Date:** 2024-10-25  
**Description:** Cat vs Dog image classification using TensorFlow CNN, demonstrating end-to-end model development, training, evaluation, and saving.

# Cat vs Dog Image Classification (TensorFlow CNN)
This project implements a convolutional neural network (CNN) in TensorFlow to classify images of cats and dogs. It demonstrates data preprocessing with augmentation, model architecture design, training, and evaluation — including accuracy/loss plots and a confusion matrix.

## Technologies
- Python
- TensorFlow
- Keras
- NumPy
- scikit-learn

## Key concepts covered
- Image preprocessing with data augmentation
- CNN architecture for binary classification
- Model training and validation
- Visualization of accuracy and loss over epochs
- Evaluation using a confusion matrix

## Imports
We import necessary libraries for building, training, evaluating, and visualizing the convolutional neural network (CNN) model.

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

For this project, the focus is on binary classification using the classic "cat and dog" problem. The goal is to train a model to distinguish between two categories: cats and dogs. A basic overview of the process is as follows:

- **Dataset:** The process begins with a labeled dataset containing images of cats and dogs, each labeled with the correct category. This dataset is used for training the model.
- **Feature Extraction:** The model analyzes features of the images (e.g., pixel values, textures, shapes, colors) to learn distinguishing characteristics that separate cats from dogs.
- **Model Training:** A machine learning algorithm — in this case, a convolutional neural network (CNN) — is trained on the dataset. The algorithm adjusts its parameters to minimize classification error.
- **Validation and Testing:** The model is evaluated on a separate set of labeled images (validation and testing datasets) to measure its performance and make adjustments.
- **Prediction:** Once trained, the model can categorize new images as either "cat" or "dog" based on what it has learned.

To explore this binary classification task, I used TensorFlow — an open-source machine learning library — to build and train a convolutional neural network (CNN) for classifying images of cats and dogs.

## Dataset Download and Preparation

We download and extract the Cats vs Dogs dataset and define directory paths for training and validation data.

In [ ]:
# Download and extract the Cats vs Dogs dataset
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

# Define base directories for dataset
base_dir = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

## Data Preprocessing

We use `ImageDataGenerator` to normalize pixel values and apply data augmentation to the training data.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Set up data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Ensure alignment for evaluation
)

## Model Architecture

We define a CNN model consisting of convolutional, pooling, and dense layers for binary classification.

In [ ]:
from tensorflow.keras import layers, models

# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

## Model Compilation and Training

We compile the model using the Adam optimizer and binary cross-entropy loss, then train it for 15 epochs.

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50
)

## Training and Validation Metrics

We visualize the training and validation accuracy and loss across epochs.

In [ ]:
# Plot accuracy and loss over epochs
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

## Saving the Model

We save the trained model to a file for future use.

In [ ]:
# Save the trained model
model.save('cats_and_dogs_classifier.h5')

## Explanation of the Code

1. **Data Preparation:** The dataset is automatically downloaded and extracted. It includes separate directories for training and validation sets of cat and dog images.
2. **Data Augmentation:** `ImageDataGenerator` is used to apply data augmentation techniques, which help improve model generalization.
3. **Model Architecture:** A CNN is built with multiple convolutional and pooling layers, followed by dense layers for binary classification.
4. **Training the Model:** The model is compiled using the Adam optimizer and binary cross-entropy loss, and trained for 15 epochs.
5. **Saving the Model:** The trained model is saved as an `.h5` file for future use.

This project demonstrates binary image classification using a convolutional neural network (CNN) to distinguish between cats and dogs. It covers dataset preparation, model training, evaluation (including accuracy/loss plots and a confusion matrix), and saving the trained model for future use.

## Evaluation: Confusion Matrix

We compute and display the confusion matrix to visualize how well the model distinguishes cats from dogs.

In [ ]:
# Generate predictions on the validation data
y_pred_prob = model.predict(validation_generator)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Get true labels
y_true = validation_generator.classes

# Compute and display the confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Cat', 'Dog'])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix: Cats vs Dogs')
plt.show()

## Evaluation: Classification Report

We compute precision, recall, F1-score, and support for each class.

In [ ]:
from sklearn.metrics import classification_report

# Print classification report
print(classification_report(y_true, y_pred, target_names=['Cat', 'Dog']))

## Evaluation: Validation Accuracy

We compute and display the overall accuracy on the validation set.

In [ ]:
from sklearn.metrics import accuracy_score

# Print validation accuracy
val_accuracy = accuracy_score(y_true, y_pred)
print(f'Validation Accuracy: {val_accuracy:.4f}')

## Conclusion

Below is a summary of the model's performance, dynamically generated based on the validation results:

In [ ]:
# Dynamically generate conclusion text based on actual accuracy
print(f"""
The CNN model achieved a validation accuracy of approximately {val_accuracy * 100:.2f}%.
The confusion matrix and classification report suggest that the model learned to distinguish between cats and dogs reasonably well.

Potential future improvements could include:
- Experimenting with deeper architectures or pre-trained models (e.g., transfer learning)
- Additional data augmentation or regularization
- Hyperparameter tuning

This notebook demonstrates the basic workflow of building, training, evaluating, and saving a TensorFlow CNN for image classification.
""")